# 1. SE(3) Transforms

SE(3) (Special Euclidean group in 3D) represents rigid-body transformations combining rotation and translation.

## Key Components

**Rotations:** Represented as 3×3 matrices, quaternions, or Euler angles.

**Translation:** Represented as 3D vectors.

## Homogeneous Matrix Representation

SE(3) transformations are commonly expressed as 4×4 homogeneous matrices:

$$T = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix}$$

Where:
- **R** is a 3×3 rotation matrix
- **t** is a 3×1 translation vector
- The bottom row [0 1] ensures homogeneous coordinates

## Applications

SE(3) transforms are fundamental in:
- Robotics (pose representation, kinematics)
- Computer vision (camera poses, object tracking)
- 3D graphics (object transformations)
- Autonomous systems (localization and mapping)

# 2. Rotation Matrices

### Theory

A rotation matrix is a 3×3 matrix that rotates points in 3D about the origin.

Key properties:

- **Orthogonal:** \(R^\top R = I\) (transpose = inverse).
- **Determinant = 1:** preserves orientation and scale.

> Conventions: matrices below follow the right-hand rule and represent active rotations (rotating points in a fixed coordinate frame).

### Rotations about principal axes

X-axis (changes Y and Z):

$$
R_x(\theta)=
\begin{bmatrix}
1 & 0 & 0\\[4pt]
0 & \cos\theta & -\sin\theta\\[4pt]
0 & \sin\theta & \cos\theta
\end{bmatrix}
$$

Y-axis (changes X and Z):

$$
R_y(\theta)=
\begin{bmatrix}
\cos\theta & 0 & \sin\theta\\[4pt]
0 & 1 & 0\\[4pt]
-\sin\theta & 0 & \cos\theta
\end{bmatrix}
$$

Z-axis (changes X and Y):

$$
R_z(\theta)=
\begin{bmatrix}
\cos\theta & -\sin\theta & 0\\[4pt]
\sin\theta & \cos\theta & 0\\[4pt]
0 & 0 & 1
\end{bmatrix}
$$

### Notes

- Composition: successive rotations are composed by matrix multiplication (order matters).
- Inverse: \(R^{-1} = R^\top\).
- Use these matrices to build a 4×4 homogeneous transform for SE(3):

$$
T = \begin{bmatrix} R & t \\[4pt] 0 & 1 \end{bmatrix}
$$

where \(R\) is a 3×3 rotation matrix and \(t\) is a 3×1 translation vector.


In [21]:
import numpy as np

# Rotation around X-axis
def Rx(theta):
    """
    Rotate points around X-axis by theta radians.
    Args:
        theta: rotation angle in radians
    Returns:
        3x3 rotation matrix
    """
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[1, 0, 0],
                  [0, c, -s],
                  [0, s, c]])
    return R

# Rotation around Y-axis
def Ry(theta):
    """
    Rotate points around Y-axis by theta radians.
    """
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, 0, s],
                  [0, 1, 0],
                  [-s, 0, c]])
    return R

# Rotation around Z-axis
def Rz(theta):
    """
    Rotate points around Z-axis by theta radians.
    """
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, -s, 0],
                  [s, c, 0],
                  [0, 0, 1]])
    return R


In [22]:
# Original point in 3D
point = np.array([1, 0, 0])

# Rotate 90 degrees around Z-axis
theta = np.pi/2
rotated_point = Rz(theta) @ point  # Matrix multiplication

print(rotated_point)


[6.123234e-17 1.000000e+00 0.000000e+00]


# 3. Euler Angles

- A sequence of 3 rotations around axes (e.g., ZYX).
- Convert Euler angles → Rotation matrix:
\[
R = R_z(\gamma) R_y(\beta) R_x(\alpha)
\]
- Convert back: extract angles from rotation matrix.
- **Problem:** Gimbal lock can occur (loss of one DOF when two axes align).


In [24]:
def euler_to_matrix(roll, pitch, yaw):
    """Convert Euler angles to rotation matrix (ZYX order)"""
    return Rz(yaw) @ Ry(pitch) @ Rx(roll)

def matrix_to_euler(R):
    """Convert rotation matrix to Euler angles (ZYX order)"""
    sy = np.sqrt(R[0,0]**2 + R[1,0]**2)
    singular = sy < 1e-6
    if not singular:
        x = np.arctan2(R[2,1], R[2,2])
        y = np.arctan2(-R[2,0], sy)
        z = np.arctan2(R[1,0], R[0,0])
    else:
        x = np.arctan2(-R[1,2], R[1,1])
        y = np.arctan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

# Test roundtrip
angles = np.array([0.1, 0.2, 0.3])
R = euler_to_matrix(*angles)
angles_back = matrix_to_euler(R)
print("Original:", angles)
print("Recovered:", angles_back)


Original: [0.1 0.2 0.3]
Recovered: [0.1 0.2 0.3]


# 4. Quaternions

A quaternion is a rotation stored as 4 numbers:

\[
q = [w, x, y, z]
\]

They avoid gimbal lock and are used in SLAM, robotics, AR/VR.

## Quaternion → Rotation Matrix
We use the formula:


$$R(q)=
\begin{bmatrix}
1-2(y^2+z^2) & 2(xy-wz) & 2(xz+wy)\\
2(xy+wz) & 1-2(x^2+z^2) & 2(yz-wx)\\
2(xz-wy) & 2(yz+wx) & 1-2(x^2+y^2)
\end{bmatrix}$$

## Rotation Matrix → Quaternion
We extract \((w,x,y,z)\) based on the matrix diagonal.


In [26]:
import numpy as np

def quat_to_matrix(q):
    """Quaternion [w, x, y, z] → 3×3 rotation matrix."""
    w, x, y, z = q
    R = np.array([
        [1 - 2*(y*y + z*z),   2*(x*y - w*z),       2*(x*z + w*y)],
        [2*(x*y + w*z),       1 - 2*(x*x + z*z),   2*(y*z - w*x)],
        [2*(x*z - w*y),       2*(y*z + w*x),       1 - 2*(x*x + y*y)]
    ])
    return R

def matrix_to_quat(R):
    """3×3 rotation matrix → Quaternion [w, x, y, z]."""
    m00, m01, m02 = R[0]
    m10, m11, m12 = R[1]
    m20, m21, m22 = R[2]

    trace = m00 + m11 + m22

    if trace > 0:
        S = np.sqrt(trace + 1.0) * 2
        w = 0.25 * S
        x = (m21 - m12) / S
        y = (m02 - m20) / S
        z = (m10 - m01) / S

    elif (m00 > m11) and (m00 > m22):
        S = np.sqrt(1.0 + m00 - m11 - m22) * 2
        w = (m21 - m12) / S
        x = 0.25 * S
        y = (m01 + m10) / S
        z = (m02 + m20) / S

    elif m11 > m22:
        S = np.sqrt(1.0 + m11 - m00 - m22) * 2
        w = (m02 - m20) / S
        x = (m01 + m10) / S
        y = 0.25 * S
        z = (m12 + m21) / S

    else:
        S = np.sqrt(1.0 + m22 - m00 - m11) * 2
        w = (m10 - m01) / S
        x = (m02 + m20) / S
        y = (m12 + m21) / S
        z = 0.25 * S

    return np.array([w, x, y, z])


In [30]:
# Test a quaternion
q = np.array([1, 1, 0, 0])  # 45° rotation around X axis

R = quat_to_matrix(q)
q_back = matrix_to_quat(R)

print("Quaternion:", q)
print("\nRotation Matrix:\n", R)
print("\nRecovered Quaternion:", q_back)


Quaternion: [1 1 0 0]

Rotation Matrix:
 [[ 1  0  0]
 [ 0 -1 -2]
 [ 0  2 -1]]

Recovered Quaternion: [1. 1. 0. 0.]


# 5. SE(3) Transform

SE(3) = rotation (R) + translation (t).

Matrix form:

$$T =
\begin{bmatrix}
R & t\\
0 & 1
\end{bmatrix}$$

To transform a point:

1. Convert point to homogeneous:  
   \([x, y, z, 1]^T\)
2. Multiply with \(T\)
3. Convert back to \((x,y,z)\)

---


In [31]:
def se3_transform(R, t):
    """Build SE(3) 4×4 transformation matrix."""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def apply_se3(T, points):
    """Apply SE(3) transform to N×3 points."""
    n = points.shape[0]
    homo = np.hstack([points, np.ones((n, 1))])
    transformed = (T @ homo.T).T
    return transformed[:, :3]


In [ ]:
R = Rz(np.pi/2)
t = np.array([1, 2, 3])

T = se3_transform(R, t)

points = np.array([
    [1, 0, 0],
    [0, 1, 0]
])

result = apply_se3(T, points)

print("SE(3) Matrix:\n", T)
print("\nOriginal Points:\n", points)
print("\nTransformed Points:\n", result)

SE(3) Matrix:
 [[ 6.123234e-17 -1.000000e+00  0.000000e+00  1.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00  2.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  3.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

Original Points:
 [[1 0 0]
 [0 1 0]]

Transformed Points:
 [[1. 3. 3.]
 [0. 2. 3.]]


### What just happened?

- I rotated the points 90° around the Z axis.
- Then I moved them by t = [1,2,3].
- The final coordinates you see are the new positions in space.


# 6. Visualizing SE(3) Transformations

We plot:

- Blue points = original
- Red points = transformed

Sliders let you rotate and translate live.


In [35]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets

def Rx(a): return np.array([[1,0,0],[0,np.cos(a),-np.sin(a)],[0,np.sin(a),np.cos(a)]])
def Ry(a): return np.array([[np.cos(a),0,np.sin(a)],[0,1,0],[-np.sin(a),0,np.cos(a)]])
def Rz(a): return np.array([[np.cos(a),-np.sin(a),0],[np.sin(a),np.cos(a),0],[0,0,1]])

points = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])

def plot_se3(rx, ry, rz, tx, ty, tz):
    R = Rx(rx) @ Ry(ry) @ Rz(rz)
    t = np.array([tx, ty, tz])
    T = se3_transform(R, t)
    transformed = apply_se3(T, points)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:,0], points[:,1], points[:,2], label='Original')
    ax.scatter(transformed[:,0], transformed[:,1], transformed[:,2], label='Transformed')
    ax.set_xlim(-3,3); ax.set_ylim(-3,3); ax.set_zlim(-3,3)
    ax.legend()
    plt.show()

widgets.interact(
    plot_se3,
    rx=(-np.pi, np.pi, 0.1),
    ry=(-np.pi, np.pi, 0.1),
    rz=(-np.pi, np.pi, 0.1),
    tx=(-2, 2, 0.1),
    ty=(-2, 2, 0.1),
    tz=(-2, 2, 0.1),
)


interactive(children=(FloatSlider(value=-0.04159265358979303, description='rx', max=3.141592653589793, min=-3.…

<function __main__.plot_se3(rx, ry, rz, tx, ty, tz)>